In [1]:
import os
import sys
import pandas as pd
import cv2
import numpy as np
import glob
import json

In [2]:
sys.path.append(os.path.abspath('../data_processing'))

In [3]:
from vtt_utils import *
from video_utils import *

## Chem Speaker

In [4]:
data_dir = "/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset"
speaker = "chem"
speaker_dir = os.path.join(data_dir, f"{speaker}")
video_dir = os.path.join(speaker_dir, "processed_videos")
label_file = os.path.join(speaker_dir, f"all_labels.txt")

# Getting all different video ids
train_video_ids = open(os.path.join(speaker_dir, "train.txt")).read().split()
val_video_ids = open(os.path.join(speaker_dir, "val.txt")).read().split()
test_video_ids = open(os.path.join(speaker_dir, "test.txt")).read().split()

In [18]:
with open(label_file) as file:
    lines = file.readlines()
    print(len(lines))
    videos_data = []
    for i, line in enumerate(lines):
        video_metadata = []
        rel_path, gt_text = line.split()[0], " ".join(line.split()[1:])
        video_id_list = os.path.basename(rel_path).split('_')
        video_id = '_'.join(video_id_list[:-3])

        video_filepath = os.path.join(data_dir, rel_path)
        cap = cv2.VideoCapture(video_filepath)
        num_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        fps = cap.get(cv2.CAP_PROP_FPS)

        num_seconds = num_frames/fps
        duration_str = seconds_to_hhmmss(num_seconds)
        print(rel_path)
        video_metadata = {
            'video_path': rel_path,
            'video_id': video_id,
            'transcript': gt_text,
            'duration': duration_str,
            'num_frames': num_frames,
            'seconds': num_seconds
        }

        videos_data.append(video_metadata)

9314
chem/processed_videos/--imKPteSwQ_0_0_0.mp4
chem/processed_videos/--imKPteSwQ_0_1_1.mp4
chem/processed_videos/--imKPteSwQ_0_2_2.mp4
chem/processed_videos/--imKPteSwQ_1_10_4.mp4
chem/processed_videos/--imKPteSwQ_1_11_5.mp4
chem/processed_videos/--imKPteSwQ_1_12_6.mp4
chem/processed_videos/--imKPteSwQ_1_9_3.mp4
chem/processed_videos/--imKPteSwQ_2_13_7.mp4
chem/processed_videos/--imKPteSwQ_2_14_8.mp4
chem/processed_videos/--imKPteSwQ_2_15_9.mp4
chem/processed_videos/--imKPteSwQ_2_16_10.mp4
chem/processed_videos/--imKPteSwQ_2_17_11.mp4
chem/processed_videos/--pzKBW13FE_0_0_0.mp4
chem/processed_videos/--pzKBW13FE_0_1_1.mp4
chem/processed_videos/--pzKBW13FE_0_2_2.mp4
chem/processed_videos/--pzKBW13FE_0_3_3.mp4
chem/processed_videos/--pzKBW13FE_0_4_4.mp4
chem/processed_videos/--pzKBW13FE_1_10_7.mp4
chem/processed_videos/--pzKBW13FE_1_8_5.mp4
chem/processed_videos/--pzKBW13FE_1_9_6.mp4
chem/processed_videos/--pzKBW13FE_2_12_8.mp4
chem/processed_videos/--pzKBW13FE_2_13_9.mp4
chem/processed

In [19]:
print(videos_data[0])
speaker_df = pd.DataFrame(videos_data)

speaker_df

{'video_path': 'chem/processed_videos/--imKPteSwQ_0_0_0.mp4', 'video_id': '--imKPteSwQ', 'transcript': 'WHEN I BURN THE HYDROCARBON BUTANE HEAT IS RELEASED', 'duration': '00:00:03.360', 'num_frames': 84.0, 'seconds': 3.36}


,video_path,video_id,transcript,duration,num_frames,seconds
0,chem/processed_videos/--imKPteSwQ_0_0_0.mp4,--imKPteSwQ,WHEN I BURN THE HYDROCARBON BUTANE HEAT IS REL...,00:00:03.360,84.0,3.36
1,chem/processed_videos/--imKPteSwQ_0_1_1.mp4,--imKPteSwQ,SO I HAVE A QUESTION FOR YOU,00:00:01.120,28.0,1.12
2,chem/processed_videos/--imKPteSwQ_0_2_2.mp4,--imKPteSwQ,IF I TAKE THREE BUTANE BALLOONS ONE WITH A TEN...,00:00:15.040,376.0,15.04
3,chem/processed_videos/--imKPteSwQ_1_10_4.mp4,--imKPteSwQ,AND IN EACH CASE OXYGEN IS IN EXCESS,00:00:03.600,90.0,3.60
4,chem/processed_videos/--imKPteSwQ_1_11_5.mp4,--imKPteSwQ,SO BUTANE LIMITS THE AMOUNT OF ENERGY RELEASED,00:00:03.560,89.0,3.56
...,...,...,...,...,...,...
9309,chem/processed_videos/zsk1S2r2r_o_7_25_19.mp4,zsk1S2r2r_o,AND THEY BOTH HAVE POSITIVE SLOPES BECAUSE FOR...,00:00:13.560,339.0,13.56
9310,chem/processed_videos/zsk1S2r2r_o_8_30_20.mp4,zsk1S2r2r_o,FOR DELTA G A NEGATIVE INTERCEPT CORRESPONDING...,00:00:06.400,160.0,6.40
9311,chem/processed_videos/zsk1S2r2r_o_8_31_21.mp4,zsk1S2r2r_o,A NEGATIVE SLOPE BECAUSE IT'S MINUS T DELTA S,00:00:04.000,100.0,4.00
9312,chem/processed_videos/zsk1S2r2r_o_9_35_22.mp4,zsk1S2r2r_o,SO NEGATIVE ENTHALPY POSITIVE SLOPE,00:00:02.640,66.0,2.64


In [20]:
print(f"{val_video_ids = }")
print(f"{test_video_ids = }")
val_videos_df = speaker_df[speaker_df['video_id'].isin(val_video_ids)]
test_videos_df = speaker_df[speaker_df['video_id'].isin(test_video_ids)]
train_videos_df = speaker_df[speaker_df['video_id'].isin(train_video_ids)]

val_video_ids = ['6DGPuhFoiJI', 'pKQgwIPlNL4', '9z0sMWVIm7A', 'DP4HnlNRR_4', 't6cGw1scLvc', 'JXHl3rYqlHU', 'vQ_ujXXGRZs', 'Xt-yom79grM', '-vJXlNkMqYE', 'jHbuu287uxc', 'DVeicfdiyQk', 'nE480ADhZsY', 'YttroH0ftiA', 'aNEDU6EL8jc', 'VY3yL2nDHuQ', 'hOwQQryixJM', 'XwD8___Mm1I']
test_video_ids = ['wrgnaV-cVrs', 'QL6TFnaXvMg', 'vC2v1n-bZQU', 'AGZZGR5Otxw', '4kxMdkzCJGA', 'VzvinAckmQU', 'lItaSuF6Uqg', 'V_0Bnc_URos', 'xcJkUZ4fzEE', 'oYHyZarcCQQ', 'U6uWvuiO1rA', 'JIKXNx5TK3Q', 'FeJ1LQe6Zh4', '1yoOSIcsCyI', 'Z08pNaGQPv8', 'hSBT6oe6dSA']


In [11]:
total_val_seconds = val_videos_df['seconds'].sum()
print(f"VAL: Duration: {seconds_to_hhmmss(total_val_seconds)} | Total seconds: {total_val_seconds}")
val_videos_df[val_videos_df['seconds'] > 19]

VAL: Duration: 00:40:13.480 | Total seconds: 2413.4799999999996


,video_path,video_id,transcript,duration,num_frames,seconds
4988,chem/processed_videos/Xt-yom79grM_2_27_18.mp4,Xt-yom79grM,SO I COULD HAVE COME AT THIS FROM EITHER DIREC...,00:00:19.600,490.0,19.60
5617,chem/processed_videos/aNEDU6EL8jc_0_1_1.mp4,aNEDU6EL8jc,HERE I HAVE A CHEMICAL REACTION THAT I'VE WRIT...,00:00:19.760,494.0,19.76
5660,chem/processed_videos/aNEDU6EL8jc_3_53_44.mp4,aNEDU6EL8jc,SO IF I DO THIS REACTION AT 25 DEGREES C I MIG...,00:00:19.520,488.0,19.52
7515,chem/processed_videos/nE480ADhZsY_5_30_23.mp4,nE480ADhZsY,POSITIVE BUT YOU CAN OVERCOME IT AS LONG AS YO...,00:00:19.400,485.0,19.40


In [12]:
total_test_seconds = test_videos_df['seconds'].sum()
print(f"TEST: Duration: {seconds_to_hhmmss(total_test_seconds)} | Total Seconds: {total_test_seconds}")
test_videos_df[test_videos_df['seconds'] > 19]

TEST: Duration: 00:36:10.160 | Total Seconds: 2170.16


,video_path,video_id,transcript,duration,num_frames,seconds
574,chem/processed_videos/1yoOSIcsCyI_3_15_9.mp4,1yoOSIcsCyI,ORBITAL TO FORM SP3 HYBRIDS FOUR EQUIVALENT OR...,00:00:19.720,493.0,19.72
927,chem/processed_videos/4kxMdkzCJGA_0_0_0.mp4,4kxMdkzCJGA,WHEN WE TALK ABOUT QUANTUM MECHANICS IN CHEM 1...,00:00:19.320,483.0,19.32


In [63]:
train_videos_df[train_videos_df['num_frames'] > 500]

,video_path,video_id,transcript,duration,num_frames,seconds
33,chem/processed_videos/-AOyf8eA1ic_0_1.mp4,-AOyf8eA1ic,THE STANDARD STATE FREE ENERGY IS DELTA H MINU...,00:00:22.080,552.0,22.08
35,chem/processed_videos/-AOyf8eA1ic_2_4.mp4,-AOyf8eA1ic,NOW I HAVE A LINEAR FUNCTION OF NATURAL LOG K ...,00:00:24.320,608.0,24.32
48,chem/processed_videos/-AOyf8eA1ic_4_20.mp4,-AOyf8eA1ic,SO IF I MEASURE EQUILIBRIUM CONSTANT AS A FUNC...,00:00:21.440,536.0,21.44
285,chem/processed_videos/0Ot2WbfIeMQ_1_13.mp4,0Ot2WbfIeMQ,BUT IF WE WERE TO TAKE A WIRE AND CONNECT THE ...,00:00:27.200,680.0,27.20
308,chem/processed_videos/0Ot2WbfIeMQ_4_45.mp4,0Ot2WbfIeMQ,SO AS NEGATIVE ELECTRONS FLOW TO THIS SIDE AND...,00:00:27.120,678.0,27.12
330,chem/processed_videos/100n-E7o9Ug_0_2.mp4,100n-E7o9Ug,IN THIS CASE FOR MOLECULES THAT WE'RE FORMING ...,00:00:22.920,573.0,22.92
360,chem/processed_videos/11QPT4cbBO8_0_2.mp4,11QPT4cbBO8,SO CHEMISTRY IS GOING TO TAKE US FROM A STUDY ...,00:00:26.120,653.0,26.12
1011,chem/processed_videos/65q2f8Naf8M_2_17.mp4,65q2f8Naf8M,IF YOU LOOK AT THIS IN TERMS OF A LINE THIS LO...,00:00:29.720,743.0,29.72
1276,chem/processed_videos/7W0cz0oGHGE_10_65.mp4,7W0cz0oGHGE,083,00:00:20.960,524.0,20.96
1451,chem/processed_videos/8pdWHCJSkso_2_15.mp4,8pdWHCJSkso,SO HERE'S A SEQUENCE OF ACIDS WHERE I'M GOING ...,00:00:26.520,663.0,26.52


In [49]:
val_videos_df[val_videos_df['num_frames'] > 500]

,video_path,video_id,transcript,duration,num_frames,seconds
3010,chem/processed_videos/JXHl3rYqlHU_2_18.mp4,JXHl3rYqlHU,IS THE TEMPERATURE IN CELSIUS PLUS 27315,00:02:06.640,3166.0,126.64
5603,chem/processed_videos/aNEDU6EL8jc_0_1.mp4,aNEDU6EL8jc,HERE I HAVE A CHEMICAL REACTION THAT I'VE WRIT...,00:00:22.000,550.0,22.00
5645,chem/processed_videos/aNEDU6EL8jc_3_53.mp4,aNEDU6EL8jc,SO IF I DO THIS REACTION AT 25 DEGREES C I MIG...,00:00:22.440,561.0,22.44
7490,chem/processed_videos/nE480ADhZsY_5_30.mp4,nE480ADhZsY,POSITIVE BUT YOU CAN OVERCOME IT AS LONG AS YO...,00:00:24.560,614.0,24.56


In [60]:
test_videos_df[test_videos_df['num_frames'] > 500]

,video_path,video_id,transcript,duration,num_frames,seconds
565,chem/processed_videos/1yoOSIcsCyI_3_15.mp4,1yoOSIcsCyI,ORBITAL TO FORM SP3 HYBRIDS FOUR EQUIVALENT OR...,00:00:23.040,576.0,23.04
918,chem/processed_videos/4kxMdkzCJGA_0_0.mp4,4kxMdkzCJGA,WHEN WE TALK ABOUT QUANTUM MECHANICS IN CHEM 1...,00:00:23.760,594.0,23.76
937,chem/processed_videos/4kxMdkzCJGA_1_23.mp4,4kxMdkzCJGA,HAS N EQUAL 1,00:01:28.960,2224.0,88.96


In [24]:
clips_dir = os.path.join(speaker_dir, f"sentence_clips")

clip_files = glob.glob(os.path.join(clips_dir, "*/clips.json"))
print(f"{len(clip_files) = }")
print(f"{clip_files = }")

len(clip_files) = 346
clip_files = ['/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/cy4vUW9x62s/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/e27ZIIsvuys/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/k3ISd1oHxKA/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/7A5UuW3EJPc/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/LPol-lH6nmI/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/yHIDWAYbYBs/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/ZFZ2J8gwIkE/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/OlehBTSyPDA/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/ZuwQf_7yXrg/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/PyIiTK0eXrA/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/oYVD8MSW8NE/clips.json', '/ssd_s

In [25]:
import json
videos_metadata = []

for clip_file_idx, clip_file in enumerate(clip_files):
    print(f"{clip_file = }")
    tracks_clips = json.load(open(clip_file))
    for track_clips in tracks_clips:
        for clip in track_clips['clips']:
            start_time = clip['start']
            end_time = clip['end']
            sentence = clip['sentence']
            video_path = clip['clip_output_path']
            video_id = os.path.basename(os.path.dirname(video_path))
            duration = end_time - start_time
            rel_path = os.path.relpath(video_path, data_dir)

            video_data = {
                'video_path': rel_path,
                'video_id': video_id,
                'start': start_time,
                'end': end_time,
                'transcript': sentence,
                'seconds': duration,
            }

            videos_metadata.append(video_data)
    # break

clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/cy4vUW9x62s/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/e27ZIIsvuys/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/k3ISd1oHxKA/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/7A5UuW3EJPc/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/LPol-lH6nmI/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/yHIDWAYbYBs/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/ZFZ2J8gwIkE/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/OlehBTSyPDA/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/ZuwQf_7yXrg/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem/sentence_clips/PyIiTK0eXrA/clips.json'
clip_file = '/ssd_sc

In [26]:
speaker_df = pd.DataFrame(videos_metadata)
speaker_df

,video_path,video_id,start,end,transcript,seconds
0,chem/sentence_clips/cy4vUW9x62s/cy4vUW9x62s_0_...,cy4vUW9x62s,0.609,3.591,Let's look at a couple molecules and see if w...,2.982
1,chem/sentence_clips/cy4vUW9x62s/cy4vUW9x62s_1_...,cy4vUW9x62s,16.079,17.220,But don't be fooled.,1.141
2,chem/sentence_clips/cy4vUW9x62s/cy4vUW9x62s_1_...,cy4vUW9x62s,17.300,27.367,"Remember, to find a linear ion or molecule, wh...",10.067
3,chem/sentence_clips/cy4vUW9x62s/cy4vUW9x62s_2_...,cy4vUW9x62s,67.751,79.124,"Now, steric number 4 and steric number 5 give...",11.373
4,chem/sentence_clips/cy4vUW9x62s/cy4vUW9x62s_2_...,cy4vUW9x62s,79.744,80.665,I have those two here.,0.921
...,...,...,...,...,...,...
9311,chem/sentence_clips/TmKL0W2skJM/TmKL0W2skJM_6_...,TmKL0W2skJM,231.196,235.638,It's the conversion factor between heat flow a...,4.442
9312,chem/sentence_clips/TmKL0W2skJM/TmKL0W2skJM_6_...,TmKL0W2skJM,236.178,238.219,Temperature changes are very easy to measure.,2.041
9313,chem/sentence_clips/TmKL0W2skJM/TmKL0W2skJM_6_...,TmKL0W2skJM,238.919,241.340,Stick a thermometer in and you can measure a t...,2.421
9314,chem/sentence_clips/TmKL0W2skJM/TmKL0W2skJM_6_...,TmKL0W2skJM,242.119,247.191,"Now, if you know the heat capacity, you can m...",5.072


In [23]:
max_seconds = speaker_df['seconds'].max()
print(list(speaker_df[speaker_df['seconds'] == max_seconds]['transcript']))
speaker_df[speaker_df['seconds'] > 20]

[" If you look at this in terms of a line, this looks like y equals mx plus b. The standard situation for a straight line has a slope determined by, in this case, delta s, and an intercept determined by delta h. So if delta s is positive, the slope of this system must be negative, because there's a negative sign there."]


,video_path,video_id,start,end,transcript,seconds
2454,chem/sentence_clips/11QPT4cbBO8/11QPT4cbBO8_0_...,11QPT4cbBO8,13.468,35.104,So chemistry is going to take us from a study ...,21.636
2501,chem/sentence_clips/65q2f8Naf8M/65q2f8Naf8M_2_...,65q2f8Naf8M,126.912,151.615,"If you look at this in terms of a line, this ...",24.703
3226,chem/sentence_clips/nE480ADhZsY/nE480ADhZsY_5_...,nE480ADhZsY,195.613,216.080,"positive, but you can overcome it as long as y...",20.467
3476,chem/sentence_clips/sY-0uDLNYmk/sY-0uDLNYmk_2_...,sY-0uDLNYmk,191.618,215.127,difference between the reactants and the prod...,23.509
4029,chem/sentence_clips/oTCKWNiWdus/oTCKWNiWdus_0_...,oTCKWNiWdus,21.902,44.040,And when you start to think about subtracting ...,22.138
4134,chem/sentence_clips/fysCY3d9zAU/fysCY3d9zAU_4_...,fysCY3d9zAU,300.804,321.398,"A k less than 1, a fraction gives you a negati...",20.594
4677,chem/sentence_clips/8pdWHCJSkso/8pdWHCJSkso_2_...,8pdWHCJSkso,73.239,95.232,So here's a sequence of acids where I'm going ...,21.993
4851,chem/sentence_clips/-AOyf8eA1ic/-AOyf8eA1ic_2_...,-AOyf8eA1ic,52.014,72.210,"Now, I have a linear function of natural log k...",20.196
6199,chem/sentence_clips/0Ot2WbfIeMQ/0Ot2WbfIeMQ_1_...,0Ot2WbfIeMQ,81.277,103.802,But if we were to take a wire and connect the...,22.525
6221,chem/sentence_clips/0Ot2WbfIeMQ/0Ot2WbfIeMQ_4_...,0Ot2WbfIeMQ,274.082,296.506,So as negative electrons flow to this side and...,22.424


In [27]:
val_videos_df = speaker_df[speaker_df['video_id'].isin(val_video_ids)]
test_videos_df = speaker_df[speaker_df['video_id'].isin(test_video_ids)]
train_videos_df = speaker_df[speaker_df['video_id'].isin(train_video_ids)]

In [28]:
train_videos_df[train_videos_df['seconds'] < 1.0].sort_values('seconds')

,video_path,video_id,start,end,transcript,seconds
3498,chem/sentence_clips/CNAU31Q7cJg/CNAU31Q7cJg_0_...,CNAU31Q7cJg,49.843,50.383,this case.,0.540
4971,chem/sentence_clips/aVRWeXk4oK8/aVRWeXk4oK8_0_...,aVRWeXk4oK8,21.571,22.191,liquid water.,0.620
6970,chem/sentence_clips/Bl-zUmeYj74/Bl-zUmeYj74_1_...,Bl-zUmeYj74,373.501,374.181,that I started in.,0.680
37,chem/sentence_clips/e27ZIIsvuys/e27ZIIsvuys_3_...,e27ZIIsvuys,105.945,106.645,visible light.,0.700
9026,chem/sentence_clips/m6SHjX_QcwE/m6SHjX_QcwE_1_...,m6SHjX_QcwE,80.652,81.352,You'd measure the rate.,0.700
...,...,...,...,...,...,...
8038,chem/sentence_clips/TZuRh_epSyI/TZuRh_epSyI_4_...,TZuRh_epSyI,127.950,128.931,How did I share them?,0.981
6238,chem/sentence_clips/AlwmlHuZRUU/AlwmlHuZRUU_1_...,AlwmlHuZRUU,127.764,128.745,This side will become green.,0.981
1507,chem/sentence_clips/3ofEcJm3U7Y/3ofEcJm3U7Y_1_...,3ofEcJm3U7Y,113.639,114.620,"And among these three,",0.981
4064,chem/sentence_clips/jzHAfhGPQxg/jzHAfhGPQxg_1_...,jzHAfhGPQxg,25.047,26.029,Delta H is negative.,0.982


In [29]:
print(val_video_ids)
val_videos_df[val_videos_df['seconds'] > 16]

['6DGPuhFoiJI', 'pKQgwIPlNL4', '9z0sMWVIm7A', 'DP4HnlNRR_4', 't6cGw1scLvc', 'JXHl3rYqlHU', 'vQ_ujXXGRZs', 'Xt-yom79grM', '-vJXlNkMqYE', 'jHbuu287uxc', 'DVeicfdiyQk', 'nE480ADhZsY', 'YttroH0ftiA', 'aNEDU6EL8jc', 'VY3yL2nDHuQ', 'hOwQQryixJM', 'XwD8___Mm1I']


,video_path,video_id,start,end,transcript,seconds
2110,chem/sentence_clips/aNEDU6EL8jc/aNEDU6EL8jc_0_...,aNEDU6EL8jc,3.332,19.755,"Here, I have a chemical reaction that I've wri...",16.423
2153,chem/sentence_clips/aNEDU6EL8jc/aNEDU6EL8jc_3_...,aNEDU6EL8jc,328.236,344.530,"So if I do this reaction at 25 degrees C, I mi...",16.294
2430,chem/sentence_clips/nE480ADhZsY/nE480ADhZsY_5_...,nE480ADhZsY,195.613,211.778,"positive, but you can overcome it as long as y...",16.165
3363,chem/sentence_clips/Xt-yom79grM/Xt-yom79grM_2_...,Xt-yom79grM,185.265,201.608,So I could have come at this from either dire...,16.343


In [30]:
val_videos_df[val_videos_df['seconds'] < 1.0]

,video_path,video_id,start,end,transcript,seconds
1051,chem/sentence_clips/vQ_ujXXGRZs/vQ_ujXXGRZs_0_...,vQ_ujXXGRZs,6.454,7.395,What do you think will form?,0.941
1400,chem/sentence_clips/t6cGw1scLvc/t6cGw1scLvc_1_...,t6cGw1scLvc,61.133,62.053,What about compression?,0.920
3353,chem/sentence_clips/Xt-yom79grM/Xt-yom79grM_2_...,Xt-yom79grM,124.457,125.238,The pressure is too high.,0.781
3364,chem/sentence_clips/Xt-yom79grM/Xt-yom79grM_2_...,Xt-yom79grM,201.708,202.689,And that's what I'm looking at.,0.981
5057,chem/sentence_clips/YttroH0ftiA/YttroH0ftiA_1_...,YttroH0ftiA,42.762,43.662,"Well, let's talk about that.",0.900
6143,chem/sentence_clips/6DGPuhFoiJI/6DGPuhFoiJI_2_...,6DGPuhFoiJI,123.580,124.301,"Now, we dilute it.",0.721
6151,chem/sentence_clips/6DGPuhFoiJI/6DGPuhFoiJI_3_...,6DGPuhFoiJI,177.318,178.119,"Now, that's interesting.",0.801
8144,chem/sentence_clips/-vJXlNkMqYE/-vJXlNkMqYE_5_...,-vJXlNkMqYE,128.059,129.000,This energy is called,0.941
9187,chem/sentence_clips/9z0sMWVIm7A/9z0sMWVIm7A_0_...,9z0sMWVIm7A,11.132,12.052,I'll go through them right now.,0.920


In [31]:
print(test_video_ids)
test_videos_df[test_videos_df['seconds'] > 16]

['wrgnaV-cVrs', 'QL6TFnaXvMg', 'vC2v1n-bZQU', 'AGZZGR5Otxw', '4kxMdkzCJGA', 'VzvinAckmQU', 'lItaSuF6Uqg', 'V_0Bnc_URos', 'xcJkUZ4fzEE', 'oYHyZarcCQQ', 'U6uWvuiO1rA', 'JIKXNx5TK3Q', 'FeJ1LQe6Zh4', '1yoOSIcsCyI', 'Z08pNaGQPv8', 'hSBT6oe6dSA']


,video_path,video_id,start,end,transcript,seconds
7467,chem/sentence_clips/1yoOSIcsCyI/1yoOSIcsCyI_3_...,1yoOSIcsCyI,101.696,118.108,"orbital to form sp3 hybrids, four equivalent o...",16.412
8646,chem/sentence_clips/4kxMdkzCJGA/4kxMdkzCJGA_0_...,4kxMdkzCJGA,1.067,17.183,When we talk about quantum mechanics in Chem 1...,16.116


In [32]:
test_videos_df[test_videos_df['seconds'] < 1.0]

,video_path,video_id,start,end,transcript,seconds
1734,chem/sentence_clips/QL6TFnaXvMg/QL6TFnaXvMg_1_...,QL6TFnaXvMg,106.434,107.394,And why would that happen?,0.960
1831,chem/sentence_clips/oYHyZarcCQQ/oYHyZarcCQQ_4_...,oYHyZarcCQQ,165.480,166.200,And that is true.,0.720
2313,chem/sentence_clips/V_0Bnc_URos/V_0Bnc_URos_2_...,V_0Bnc_URos,102.735,103.675,Lower the temperature.,0.940
5624,chem/sentence_clips/FeJ1LQe6Zh4/FeJ1LQe6Zh4_3_...,FeJ1LQe6Zh4,124.222,124.962,So let's look at,0.740
7596,chem/sentence_clips/hSBT6oe6dSA/hSBT6oe6dSA_3_...,hSBT6oe6dSA,96.739,97.579,That'll give you your 500 bolts.,0.840
8259,chem/sentence_clips/lItaSuF6Uqg/lItaSuF6Uqg_0_...,lItaSuF6Uqg,62.726,63.426,It looks kind of like,0.700
8377,chem/sentence_clips/Z08pNaGQPv8/Z08pNaGQPv8_1_...,Z08pNaGQPv8,95.137,96.078,They like to be parallel.,0.941
8391,chem/sentence_clips/Z08pNaGQPv8/Z08pNaGQPv8_2_...,Z08pNaGQPv8,163.355,164.056,Let's look at that.,0.701
8396,chem/sentence_clips/Z08pNaGQPv8/Z08pNaGQPv8_2_...,Z08pNaGQPv8,183.905,184.665,The first one,0.760
8425,chem/sentence_clips/Z08pNaGQPv8/Z08pNaGQPv8_9_...,Z08pNaGQPv8,312.455,313.376,They're wave-like.,0.921


## EH Speaker

In [4]:
data_dir = "/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset"
speaker = "eh"
speaker_dir = os.path.join(data_dir, f"{speaker}")
video_dir = os.path.join(speaker_dir, "processed_videos")
label_file = os.path.join(speaker_dir, f"all_labels.txt")

# Getting all different video ids
train_video_ids = open(os.path.join(speaker_dir, "train.txt")).read().split()
val_video_ids = open(os.path.join(speaker_dir, "val.txt")).read().split()
test_video_ids = open(os.path.join(speaker_dir, "test.txt")).read().split()

In [9]:
with open(label_file) as file:
    lines = file.readlines()
    videos_data = []
    for i, line in enumerate(lines):
        video_metadata = []
        rel_path, gt_text = line.split()[0], " ".join(line.split()[1:])
        video_id_list = os.path.basename(rel_path).split('_')
        video_id = '_'.join(video_id_list[:-3])

        video_filepath = os.path.join(data_dir, rel_path)
        cap = cv2.VideoCapture(video_filepath)
        num_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        fps = cap.get(cv2.CAP_PROP_FPS)

        num_seconds = num_frames/fps
        duration_str = seconds_to_hhmmss(num_seconds)
        # print(rel_path, video_id_list, video_id, fps)
        video_metadata = {
            'video_path': rel_path,
            'video_id': video_id,
            'transcript': gt_text,
            'duration': duration_str,
            'num_frames': num_frames,
            'seconds': num_seconds
        }

        videos_data.append(video_metadata)

In [10]:
speaker_df = pd.DataFrame(videos_data)
speaker_df

,video_path,video_id,transcript,duration,num_frames,seconds
0,eh/processed_videos/-tfNAI_PwwQ_0_0_0.mp4,-tfNAI_PwwQ,AH LET US CONTINUE WITH OUR DISCUSSION ON ROUT...,00:00:08.400,210.0,8.40
1,eh/processed_videos/-tfNAI_PwwQ_0_1_1.mp4,-tfNAI_PwwQ,AH IN THE LAST LECTURE WE TALKED ABOUT ROUTING...,00:00:14.600,365.0,14.60
2,eh/processed_videos/-tfNAI_PwwQ_1_100_108.mp4,-tfNAI_PwwQ,AND ONCE THIS IS DONE THE ROUTING TABLES ARE U...,00:00:10.920,273.0,10.92
3,eh/processed_videos/-tfNAI_PwwQ_1_101_109.mp4,-tfNAI_PwwQ,SUPPOSE I AM A ROUTER I AM A ROUTER AND I HAVE...,00:00:10.960,274.0,10.96
4,eh/processed_videos/-tfNAI_PwwQ_1_102_110.mp4,-tfNAI_PwwQ,THAT WHERE TO FORWARD IT NEXT WHICH IS THE NEX...,00:00:19.160,479.0,19.16
...,...,...,...,...,...,...
5251,eh/processed_videos/xMubwlXSJf8_4_157_167.mp4,xMubwlXSJf8,WHERE WE HAVE TALKED ABOUT SOME OF THE COMMONL...,00:00:09.440,236.0,9.44
5252,eh/processed_videos/xMubwlXSJf8_4_158_168.mp4,xMubwlXSJf8,WELL YOU ALSO USE THEM EITHER KNOWINGLY OR UNK...,00:00:19.360,484.0,19.36
5253,eh/processed_videos/xMubwlXSJf8_4_159_169.mp4,xMubwlXSJf8,SO THESE FEATURES ALLOW USERS AND APPLICATIONS...,00:00:19.800,495.0,19.80
5254,eh/processed_videos/xMubwlXSJf8_4_160_170.mp4,xMubwlXSJf8,SO IN THE NEXT FEW LECTURES WE SHALL BE LOOKIN...,00:00:12.080,302.0,12.08


In [11]:
print(f"{val_video_ids = }")
print(f"{test_video_ids = }")
val_videos_df = speaker_df[speaker_df['video_id'].isin(val_video_ids)]
test_videos_df = speaker_df[speaker_df['video_id'].isin(test_video_ids)]
train_videos_df = speaker_df[speaker_df['video_id'].isin(train_video_ids)]

val_video_ids = ['QhTuW5M2t6c']
test_video_ids = ['jB_dXbOKppI', 'xMubwlXSJf8']


In [8]:
max_val_frames = val_videos_df['num_frames'].max()
print(f"Max number of frames in val = {max_val_frames}")
val_videos_df[val_videos_df['num_frames'] > 500]

Max number of frames in val = nan


,video_path,video_id,transcript,duration,num_frames,seconds


In [9]:
max_test_frames = test_videos_df['num_frames'].max()
print(f"Max number of frames in test: {max_test_frames}")
test_videos_df[test_videos_df['num_frames'] > 500]

Max number of frames in test: nan


,video_path,video_id,transcript,duration,num_frames,seconds


In [6]:
clips_dir = os.path.join(speaker_dir, f"sentence_clips")

clip_files = glob.glob(os.path.join(clips_dir, "*/clips.json"))
print(f"{len(clip_files) = }")
print(f"{clip_files = }")

len(clip_files) = 35
clip_files = ['/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/jB_dXbOKppI/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/ivsDWxQ5rDg/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/mm4fzI5ejUE/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/8F9qa4z4sHs/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/o48kpdQ5KFg/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/fJVDE9D8PKQ/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/n8VCzsv5gi4/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/37BFoKpn4xg/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/-tfNAI_PwwQ/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/cHG1a5-xX9A/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/ifiFTkyrUIA/clips.json', '/ssd_scratch/cvit/vanshg/Lip2

In [7]:
import json
videos_metadata = []

for clip_file_idx, clip_file in enumerate(clip_files):
    print(f"{clip_file = }")
    tracks_clips = json.load(open(clip_file))
    for track_clips in tracks_clips:
        for clip in track_clips['clips']:
            start_time = clip['start']
            end_time = clip['end']
            sentence = clip['sentence']
            video_path = clip['clip_output_path']
            video_id = os.path.basename(os.path.dirname(video_path))
            duration = end_time - start_time
            rel_path = os.path.relpath(video_path, data_dir)

            video_data = {
                'video_path': rel_path,
                'video_id': video_id,
                'start': start_time,
                'end': end_time,
                'transcript': sentence,
                'seconds': duration,
            }

            videos_metadata.append(video_data)
    # break

clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/jB_dXbOKppI/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/ivsDWxQ5rDg/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/mm4fzI5ejUE/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/8F9qa4z4sHs/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/o48kpdQ5KFg/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/fJVDE9D8PKQ/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/n8VCzsv5gi4/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/37BFoKpn4xg/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/-tfNAI_PwwQ/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/sentence_clips/cHG1a5-xX9A/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Li

In [8]:
clips_df = pd.DataFrame(videos_metadata)
clips_df

,video_path,video_id,start,end,transcript,seconds
0,eh/sentence_clips/jB_dXbOKppI/jB_dXbOKppI_0_0_...,jB_dXbOKppI,15.385,23.111,In the last lecture if you recall we were disc...,7.726
1,eh/sentence_clips/jB_dXbOKppI/jB_dXbOKppI_1_3_...,jB_dXbOKppI,44.655,46.857,a public key kind of an algorithm.,2.202
2,eh/sentence_clips/jB_dXbOKppI/jB_dXbOKppI_1_4_...,jB_dXbOKppI,48.117,56.964,And lastly we shall be briefly talking about s...,8.847
3,eh/sentence_clips/jB_dXbOKppI/jB_dXbOKppI_1_5_...,jB_dXbOKppI,59.266,62.388,"So, Diffie-Hellman key exchange is a",3.122
4,eh/sentence_clips/jB_dXbOKppI/jB_dXbOKppI_1_6_...,jB_dXbOKppI,63.837,69.780,very conceptually simple method for exchangin...,5.943
...,...,...,...,...,...,...
5256,eh/sentence_clips/5kw-999agbo/5kw-999agbo_4_75...,5kw-999agbo,982.741,998.283,So at present MD5 is not that widely used in s...,15.542
5257,eh/sentence_clips/5kw-999agbo/5kw-999agbo_4_75...,5kw-999agbo,998.903,1004.124,hash values or digest of various different siz...,5.221
5258,eh/sentence_clips/5kw-999agbo/5kw-999agbo_4_76...,5kw-999agbo,1009.366,1013.411,"And there is another class called RIP EMD, th...",4.045
5259,eh/sentence_clips/5kw-999agbo/5kw-999agbo_5_77...,5kw-999agbo,1022.684,1024.125,on cryptographic hash function.,1.441


In [9]:
clips_df[clips_df['seconds'] < 1].sort_values('seconds')

,video_path,video_id,start,end,transcript,seconds
2192,eh/sentence_clips/HRw1ORc7Hz0/HRw1ORc7Hz0_4_65...,HRw1ORc7Hz0,1335.150,1335.651,attackers,0.501
3770,eh/sentence_clips/M8DWk0mJQDY/M8DWk0mJQDY_2_39...,M8DWk0mJQDY,506.520,507.021,be signed,0.501
4386,eh/sentence_clips/d68lt1-EZzI/d68lt1-EZzI_1_11...,d68lt1-EZzI,1540.603,1541.124,computers.,0.521
1330,eh/sentence_clips/cHG1a5-xX9A/cHG1a5-xX9A_2_68...,cHG1a5-xX9A,1006.720,1007.260,let me repeat,0.540
4501,eh/sentence_clips/nBLAgd-oWNQ/nBLAgd-oWNQ_2_78...,nBLAgd-oWNQ,996.284,996.824,factorial.,0.540
...,...,...,...,...,...,...
4003,eh/sentence_clips/br9RRwA918U/br9RRwA918U_2_13...,br9RRwA918U,1249.215,1250.196,and this sec was 100.,0.981
2249,eh/sentence_clips/m582hVNdFFk/m582hVNdFFk_2_52...,m582hVNdFFk,508.136,509.117,This is how it works.,0.981
1351,eh/sentence_clips/cHG1a5-xX9A/cHG1a5-xX9A_2_82...,cHG1a5-xX9A,1220.114,1221.095,valid signature or not,0.981
954,eh/sentence_clips/37BFoKpn4xg/37BFoKpn4xg_2_30...,37BFoKpn4xg,339.707,340.688,with an IP address.,0.981


In [85]:
print(f"{val_video_ids = }")
print(f"{test_video_ids = }")
val_clips_df = clips_df[clips_df['video_id'].isin(val_video_ids)]
test_clips_df = clips_df[clips_df['video_id'].isin(test_video_ids)]
train_clips_df = clips_df[clips_df['video_id'].isin(train_video_ids)]

val_video_ids = ['QhTuW5M2t6c']
test_video_ids = ['jB_dXbOKppI', 'xMubwlXSJf8']


In [90]:
total_val_seconds = val_clips_df['seconds'].sum()
print(f"{total_val_seconds = } | Duration: {seconds_to_hhmmss(total_val_seconds)}")
val_clips_df[val_clips_df['seconds'] < 1.0]

total_val_seconds = 1357.9239999999993 | Duration: 00:22:37.924


,video_path,video_id,start,end,transcript,seconds
4986,eh/sentence_clips/QhTuW5M2t6c/QhTuW5M2t6c_1_4_...,QhTuW5M2t6c,53.641,54.202,various others.,0.561
4995,eh/sentence_clips/QhTuW5M2t6c/QhTuW5M2t6c_1_13...,QhTuW5M2t6c,128.365,129.126,Let us look at it.,0.761
5005,eh/sentence_clips/QhTuW5M2t6c/QhTuW5M2t6c_3_23...,QhTuW5M2t6c,214.529,215.429,This is steganography.,0.900
5017,eh/sentence_clips/QhTuW5M2t6c/QhTuW5M2t6c_3_35...,QhTuW5M2t6c,310.879,311.580,This is the idea.,0.701
5032,eh/sentence_clips/QhTuW5M2t6c/QhTuW5M2t6c_3_48...,QhTuW5M2t6c,443.109,443.829,"So, what happened?",0.720
5034,eh/sentence_clips/QhTuW5M2t6c/QhTuW5M2t6c_3_50...,QhTuW5M2t6c,452.454,453.335,"So, what was done?",0.881
5037,eh/sentence_clips/QhTuW5M2t6c/QhTuW5M2t6c_3_53...,QhTuW5M2t6c,464.465,465.226,"So, what was done?",0.761
5133,eh/sentence_clips/QhTuW5M2t6c/QhTuW5M2t6c_3_14...,QhTuW5M2t6c,1288.790,1289.410,transformation.,0.620
5137,eh/sentence_clips/QhTuW5M2t6c/QhTuW5M2t6c_3_14...,QhTuW5M2t6c,1334.810,1335.511,transformation.,0.701


In [89]:
total_test_seconds = test_clips_df['seconds'].sum()
print(f"{total_test_seconds = } | Duration: {seconds_to_hhmmss(total_test_seconds)}")
test_clips_df[test_clips_df['seconds'] < 1.0]

total_test_seconds = 2326.0520000000015 | Duration: 00:38:46.052


,video_path,video_id,start,end,transcript,seconds
1815,eh/sentence_clips/xMubwlXSJf8/xMubwlXSJf8_2_13...,xMubwlXSJf8,1342.705,1343.305,IP addresses.,0.6


## HS Speaker

In [4]:
data_dir = "/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset"
speaker = "hs"
speaker_dir = os.path.join(data_dir, f"{speaker}")
video_dir = os.path.join(speaker_dir, "processed_videos")
label_file = os.path.join(speaker_dir, f"all_labels.txt")

# Getting all different video ids
train_video_ids = open(os.path.join(speaker_dir, "train.txt")).read().split()
val_video_ids = open(os.path.join(speaker_dir, "val.txt")).read().split()
test_video_ids = open(os.path.join(speaker_dir, "test.txt")).read().split()

In [10]:
clips_dir = os.path.join(speaker_dir, f"sentence_clips")

clip_files = glob.glob(os.path.join(clips_dir, "*/clips.json"))
print(f"{len(clip_files) = }")
print(f"{clip_files = }")

len(clip_files) = 60
clip_files = ['/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/I87858eRsOM/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/Da8ylfOz-as/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/TW325abOpqA/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/yTZ24HcNYnA/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/MhX6EfYokjE/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/9zNhmqYav9o/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/KL34CAIZdqk/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/wc-B1TlhIK0/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/g-UIIH9ndto/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/kkR76LbNHRg/clips.json', '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/w1xEDRIKwOc/clips.json', '/ssd_scratch/cvit/vanshg/Lip2

In [11]:
import json
videos_metadata = []

for clip_file_idx, clip_file in enumerate(clip_files):
    print(f"{clip_file = }")
    tracks_clips = json.load(open(clip_file))
    for track_clips in tracks_clips:
        for clip in track_clips['clips']:
            start_time = clip['start']
            end_time = clip['end']
            sentence = clip['sentence']
            video_path = clip['clip_output_path']
            video_id = os.path.basename(os.path.dirname(video_path))
            duration = end_time - start_time
            rel_path = os.path.relpath(video_path, data_dir)

            video_data = {
                'video_path': rel_path,
                'video_id': video_id,
                'start': start_time,
                'end': end_time,
                'transcript': sentence,
                'seconds': duration,
            }

            videos_metadata.append(video_data)
    # break

clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/I87858eRsOM/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/Da8ylfOz-as/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/TW325abOpqA/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/yTZ24HcNYnA/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/MhX6EfYokjE/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/9zNhmqYav9o/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/KL34CAIZdqk/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/wc-B1TlhIK0/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/g-UIIH9ndto/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/hs/sentence_clips/kkR76LbNHRg/clips.json'
clip_file = '/ssd_scratch/cvit/vanshg/Li

In [12]:
clips_df = pd.DataFrame(videos_metadata)
print(f"{val_video_ids = }")
print(f"{test_video_ids = }")
val_clips_df = clips_df[clips_df['video_id'].isin(val_video_ids)]
test_clips_df = clips_df[clips_df['video_id'].isin(test_video_ids)]
train_clips_df = clips_df[clips_df['video_id'].isin(train_video_ids)]

clips_df

val_video_ids = ['I_rCxwIMnj4', 'Ue_iB4nfe6c']
test_video_ids = ['MQUx4Qs9APM', '4Fy7K7xxtHU']


,video_path,video_id,start,end,transcript,seconds
0,hs/sentence_clips/I87858eRsOM/I87858eRsOM_0_0_...,I87858eRsOM,15.694,17.815,"So, welcome to this class on hardware security.",2.121
1,hs/sentence_clips/I87858eRsOM/I87858eRsOM_1_14...,I87858eRsOM,136.329,143.655,We will be trying to estimate very accurately...,7.326
2,hs/sentence_clips/I87858eRsOM/I87858eRsOM_1_15...,I87858eRsOM,143.695,149.099,That is why the final attack can be done with ...,5.404
3,hs/sentence_clips/I87858eRsOM/I87858eRsOM_1_16...,I87858eRsOM,149.699,153.963,And it can threaten countermeasures because ma...,4.264
4,hs/sentence_clips/I87858eRsOM/I87858eRsOM_1_17...,I87858eRsOM,154.643,166.813,Assumption that the attacker will not have ac...,12.170
...,...,...,...,...,...,...
12652,hs/sentence_clips/CB-XkLse2O4/CB-XkLse2O4_6_22...,CB-XkLse2O4,1884.621,1895.426,I mean and you can verify this very easily wit...,10.805
12653,hs/sentence_clips/CB-XkLse2O4/CB-XkLse2O4_6_22...,CB-XkLse2O4,1897.714,1907.402,"So, this kind of depicted here that you can u...",9.688
12654,hs/sentence_clips/CB-XkLse2O4/CB-XkLse2O4_6_23...,CB-XkLse2O4,1909.344,1924.457,"So, now,what is again you know like because of...",15.113
12655,hs/sentence_clips/CB-XkLse2O4/CB-XkLse2O4_6_23...,CB-XkLse2O4,1925.117,1937.350,"So, what we will see is the next right we is h...",12.233


In [18]:
total_val_seconds = val_clips_df['seconds'].sum()
print(f"{total_val_seconds = } | Duration: {seconds_to_hhmmss(total_val_seconds)}")
val_clips_df[val_clips_df['seconds'] < 1.0].sort_values('seconds')

total_val_seconds = 2034.568999999999 | Duration: 00:33:54.569


,video_path,video_id,start,end,transcript,seconds
4718,hs/sentence_clips/Ue_iB4nfe6c/Ue_iB4nfe6c_1_18...,Ue_iB4nfe6c,1244.607,1245.127,done ok.,0.520
3653,hs/sentence_clips/I_rCxwIMnj4/I_rCxwIMnj4_13_1...,I_rCxwIMnj4,1095.392,1096.013,these two sets.,0.621
4694,hs/sentence_clips/Ue_iB4nfe6c/Ue_iB4nfe6c_1_16...,Ue_iB4nfe6c,1040.546,1041.267,On the y axis,0.721
3576,hs/sentence_clips/I_rCxwIMnj4/I_rCxwIMnj4_2_24...,I_rCxwIMnj4,212.827,213.568,"So, we basically",0.741
4684,hs/sentence_clips/Ue_iB4nfe6c/Ue_iB4nfe6c_1_15...,Ue_iB4nfe6c,969.835,970.656,"So, there is a trade off.",0.821
4663,hs/sentence_clips/Ue_iB4nfe6c/Ue_iB4nfe6c_1_13...,Ue_iB4nfe6c,838.905,839.766,"So, there is some squaring",0.861
4589,hs/sentence_clips/Ue_iB4nfe6c/Ue_iB4nfe6c_1_59...,Ue_iB4nfe6c,406.762,407.643,How many clock cycles?,0.881
4536,hs/sentence_clips/Ue_iB4nfe6c/Ue_iB4nfe6c_1_5_...,Ue_iB4nfe6c,43.403,44.344,"So, that was an example.",0.941


In [17]:
total_test_seconds = test_clips_df['seconds'].sum()
print(f"{total_test_seconds = } | Duration: {seconds_to_hhmmss(total_test_seconds)}")
test_clips_df[test_clips_df['seconds'] < 1.0].sort_values('seconds')

total_test_seconds = 2204.549 | Duration: 00:36:44.549


,video_path,video_id,start,end,transcript,seconds
3128,hs/sentence_clips/4Fy7K7xxtHU/4Fy7K7xxtHU_1_89...,4Fy7K7xxtHU,838.057,838.118,ok.,0.061
3978,hs/sentence_clips/MQUx4Qs9APM/MQUx4Qs9APM_1_88...,MQUx4Qs9APM,687.639,688.039,secret key.,0.400
3122,hs/sentence_clips/4Fy7K7xxtHU/4Fy7K7xxtHU_1_84...,4Fy7K7xxtHU,780.557,781.097,conflicting,0.540
3139,hs/sentence_clips/4Fy7K7xxtHU/4Fy7K7xxtHU_1_99...,4Fy7K7xxtHU,938.550,939.111,has become E.,0.561
3116,hs/sentence_clips/4Fy7K7xxtHU/4Fy7K7xxtHU_1_78...,4Fy7K7xxtHU,722.139,722.900,your eviction set.,0.761
3130,hs/sentence_clips/4Fy7K7xxtHU/4Fy7K7xxtHU_1_91...,4Fy7K7xxtHU,844.842,845.643,"So, how do you know that?",0.801
3167,hs/sentence_clips/4Fy7K7xxtHU/4Fy7K7xxtHU_3_13...,4Fy7K7xxtHU,1230.063,1230.883,design criteria.,0.820
4033,hs/sentence_clips/MQUx4Qs9APM/MQUx4Qs9APM_1_14...,MQUx4Qs9APM,1067.518,1068.479,"So, we know the value of L ok.",0.961
4026,hs/sentence_clips/MQUx4Qs9APM/MQUx4Qs9APM_1_13...,MQUx4Qs9APM,1023.399,1024.380,"So, this is the XOR of",0.981


In [16]:
train_clips_df[train_clips_df['seconds'] < 0.5]

,video_path,video_id,start,end,transcript,seconds
387,hs/sentence_clips/Da8ylfOz-as/Da8ylfOz-as_3_20...,Da8ylfOz-as,1780.453,1780.874,window of,0.421
553,hs/sentence_clips/TW325abOpqA/TW325abOpqA_2_15...,TW325abOpqA,1409.945,1410.045,ok.,0.100
598,hs/sentence_clips/yTZ24HcNYnA/yTZ24HcNYnA_2_13...,yTZ24HcNYnA,124.801,125.002,Alice.,0.201
1139,hs/sentence_clips/9zNhmqYav9o/9zNhmqYav9o_2_44...,9zNhmqYav9o,347.545,347.925,causes.,0.380
1181,hs/sentence_clips/9zNhmqYav9o/9zNhmqYav9o_2_82...,9zNhmqYav9o,746.805,746.905,ok.,0.100
1261,hs/sentence_clips/9zNhmqYav9o/9zNhmqYav9o_2_15...,9zNhmqYav9o,1441.969,1442.329,exponent.,0.360
1418,hs/sentence_clips/KL34CAIZdqk/KL34CAIZdqk_3_10...,KL34CAIZdqk,932.291,932.371,ok.,0.080
1636,hs/sentence_clips/wc-B1TlhIK0/wc-B1TlhIK0_3_14...,wc-B1TlhIK0,1294.062,1294.543,operations.,0.481
2433,hs/sentence_clips/w1xEDRIKwOc/w1xEDRIKwOc_3_14...,w1xEDRIKwOc,1366.809,1366.909,ok.,0.100
2439,hs/sentence_clips/w1xEDRIKwOc/w1xEDRIKwOc_3_14...,w1xEDRIKwOc,1427.288,1427.649,d k .,0.361
